# Introduction
This is a test for doing some basic ETL on data.

## The Goal
The goal of this task is to take a pair excel files, load them into SQL, and then transform the dataset into a separate schema (called FHIR).

### Input data
The input data is an pair of excel files with patient information in it. Each file represents an extract of a data system, each made a month apart. Each file includes patients discharged during the last two months, so the data will have intersecting data points, but the union of the two is really what you want.  In the boilerplate code below, you can see the file paths and open them up in excel to browse it if you'd like. Feel free to ask questions about the file formatting.

# Exercise

## Load into SQL
First, begin by loading the data into a SQL database. We have Postgres running for you already, so use that. 
Username: carta
Password: password

The data should be a union of the data in both extract files, with an update time corresponding to the newest update date available for each row.

## Transform into FHIR

After the data is loaded into SQL, you'll produce some FHIR resources from the data. In particular, there are two resources you'll make:

1) Patient
2) Encounter

Both of those are defined on the FHIR website, here: http://fhir.org. We will do a validation on the data once it's done!


In [1]:
import pandas as pd
import numpy as np
from carta_interview import Datasets, get_data_file
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [2]:
patient_extract1 = get_data_file(Datasets.PATIENT_EXTRACT1)
patient_extract2 = get_data_file(Datasets.PATIENT_EXTRACT2)

In [17]:
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="carta",
    password="password")

cur = conn.cursor()

table_sql = """CREATE TABLE IF NOT EXISTS fhir (
  encounter_id integer PRIMARY KEY ,
  first_name varchar(45) NOT NULL,
  last_name varchar(450) NOT NULL,
  birth_date DATE NOT NULL,
  admission_date DATE NOT NULL,
  discharge_date DATE NOT NULL,
  update_date DATE NOT NULL
)
"""
cur.execute(table_sql)

None
('public', 'fhir')
None


In [52]:
excel_1 = pd.read_excel(patient_extract1)
excel_2 = pd.read_excel(patient_extract2)

all_records = pd.concat([excel_1, excel_2])

sql = """INSERT INTO fhir values"""

new_dataset = {}
for index, record in all_records.head().iterrows():
    if record["Encounter ID"] in new_dataset:
        if record["Update D/T"] > new_dataset[record["Encounter ID"]][6]:
            new_dataset[record["Encounter ID"]][6] = record["Update D/T"]
    else:
        new_dataset[record["Encounter ID"]] = [record["Encounter ID"], record["First Name"], record["Last Name"], record["Birth Date"], record["Admission D/T"], record["Discharge D/T"], record["Update D/T"]]

        # TODO Fix below code to update rocord in DB
for record in new_dataset.values():
    print(record)
    print(sql)
    sql = sql + " (" + str(record[0]) + ", '" + record[1] + "', '" + record[2] + "', '" + record[3] + "', '" + record[4] + "', '" + record[5] + "', '" + record[6] + "')"
    #print(record["Encounter ID"], record["First Name"], record["Last Name"], record["Birth Date"], record["Admission D/T"], record["Discharge D/T"], record["Update D/T"])
    print(sql)
    cur.execute(sql)
conn.close

[1234, 'John', 'Doe', '01/02/1999', '04/12/2002 5:00 PM', '04/13/2002 10:00 PM', '04/24/2002 6:00 AM']
INSERT INTO fhir values
INSERT INTO fhir values (1234, 'John', 'Doe', '01/02/1999', '04/12/2002 5:00 PM', '04/13/2002 10:00 PM', '04/24/2002 6:00 AM')


InternalError: current transaction is aborted, commands ignored until end of transaction block
